In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install torch 
!pip install transformers
!pip install datasets 
!pip install captum

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# REFERENCIAS: 

# paper captum library: https://arxiv.org/pdf/2009.07896.pdf 
# paper Integrated Gradients: https://arxiv.org/pdf/1703.01365.pdf


In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd 
import numpy as np 

import captum
from captum.attr import visualization as viz
from captum.attr import LayerConductance, LayerIntegratedGradients

from datasets import Dataset
import torch
import torch.nn as nn


In [5]:
print(captum.__version__)

0.6.0


In [6]:
# load test 
df_test = pd.read_xml("/content/drive/MyDrive/ICMC/Introducao-PLN/atividades/exercicio-03/project_embeddings/dados_test_tratados.xml")
df_test.head()

,text,label
0,o cachorro caramelo está assistindo um cachorr...,0
1,o cara está fazendo exercícios no chão [sep] u...,1
2,um cachorro grande e um cachorro pequenino est...,1
3,um menino jovem vestindo um traje de banho ver...,0
4,um cara velho com uma barba que é cinza está a...,1


In [7]:
print(torch.cuda.is_available())

True


In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [9]:
# Carregando o tokenizer e o modelo do BERT pré-treinado em português
tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-large-portuguese-cased')
# carrega modelo 
model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/ICMC/Introducao-PLN/atividades/exercicio-03/project_embeddings/bertimbau-large-fine-tune')
model.to(device)
model.eval()
model.zero_grad()

In [10]:
model.device

device(type='cuda', index=0)

In [11]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(29794, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,

In [12]:
# text to inference 
text = 'Uma garrafa está sendo lambida pelo gato. O gato está lambendo um objeto'
# tokeniza dados
inputs = tokenizer(text, truncation=True, padding='max_length', max_length=128, return_tensors='pt')

In [13]:
inputs

{'input_ids': tensor([[  101,  1431, 16317,  1165,   698,   660, 19068,  3301,   285,   423,
         15997,   119,   231, 15997,   698, 19068, 21954,   222,  4947,   102,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

In [14]:
inputs = inputs.to(device)

In [15]:
model.bert.embeddings.word_embeddings

Embedding(29794, 1024, padding_idx=0)

In [16]:
# Faz a inferência do modelo
outputs = model(**inputs)[0]
prediction = torch.argmax(outputs).item()
print("Prediction:", prediction)

# Define a função de linha de base como uma sequência de zeros
baseline = torch.zeros_like(inputs['input_ids'])

Prediction: 1


In [17]:
# Define model output
def model_output(inputs):
  return model(inputs)[0]

# Define model input
model_input = model.bert.embeddings

In [18]:
# Integrated Gradients 
lig = LayerIntegratedGradients(model_output, model_input)

In [19]:
def construct_input_and_baseline(text):

    max_length = 128
    baseline_token_id = tokenizer.pad_token_id 
    sep_token_id = tokenizer.sep_token_id 
    cls_token_id = tokenizer.cls_token_id 

    text_ids = tokenizer.encode(text, max_length=max_length, truncation=True, add_special_tokens=False)
   
    input_ids = [cls_token_id] + text_ids + [sep_token_id]
    token_list = tokenizer.convert_ids_to_tokens(input_ids)

    baseline_input_ids = [cls_token_id] + [baseline_token_id] * len(text_ids) + [sep_token_id]
    return torch.tensor([input_ids], device=device), torch.tensor([baseline_input_ids], device=device), token_list

text = 'Uma garrafa está sendo lambida pelo gato. O gato está lambendo um objeto'
input_ids, baseline_input_ids, all_tokens = construct_input_and_baseline(text)

print(f'original text: {input_ids}')
print(f'baseline text: {baseline_input_ids}')

original text: tensor([[  101,  1431, 16317,  1165,   698,   660, 19068,  3301,   285,   423,
         15997,   119,   231, 15997,   698, 19068, 21954,   222,  4947,   102]],
       device='cuda:0')
baseline text: tensor([[101,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0, 102]], device='cuda:0')


In [20]:
input_ids.shape, baseline_input_ids.shape

(torch.Size([1, 20]), torch.Size([1, 20]))

In [21]:
# Faz a inferência do modelo
outputs = model(**inputs)
logits = outputs[0]

# Define o target para a classe prevista
prediction = torch.argmax(logits).item()
target = torch.tensor(prediction).unsqueeze(0)

In [22]:
attributions, delta = lig.attribute(inputs= input_ids,
                                    baselines= baseline_input_ids,
                                    target=target,
                                    return_convergence_delta=True,
                                    internal_batch_size=1
                                    )
print(attributions.size())

torch.Size([1, 20, 1024])


In [23]:
# sumarizar atribuicoes 
def summarize_attributions(attributions):

    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    
    return attributions

attributions_sum = summarize_attributions(attributions)
print(attributions_sum.size())

torch.Size([20])


In [24]:
from captum.attr import visualization as viz

score_vis = viz.VisualizationDataRecord(
                        word_attributions = attributions_sum,
                        pred_prob = torch.max(model(input_ids)[0]),
                        pred_class = torch.argmax(model(input_ids)[0]).cpu().numpy(),
                        true_class = 1,
                        attr_class = text,
                        attr_score = attributions_sum.sum(),       
                        raw_input_ids = all_tokens,
                        convergence_score = delta)

In [25]:
viz.visualize_text([score_vis])

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,1 (2.09),Uma garrafa está sendo lambida pelo gato. O gato está lambendo um objeto,-0.87,[CLS] Uma garra ##fa está sendo lam ##bi ##da pelo gato . O gato está lam ##bendo um objeto [SEP]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,1 (2.09),Uma garrafa está sendo lambida pelo gato. O gato está lambendo um objeto,-0.87,[CLS] Uma garra ##fa está sendo lam ##bi ##da pelo gato . O gato está lam ##bendo um objeto [SEP]


In [26]:
# funcao de interpretacao  
def interpret_text(text, true_class):

    input_ids, baseline_input_ids, all_tokens = construct_input_and_baseline(text)
    attributions, delta = lig.attribute(inputs= input_ids,
                                    baselines= baseline_input_ids,
                                    target=true_class,
                                    return_convergence_delta=True,
                                    internal_batch_size=1
                                    )
    attributions_sum = summarize_attributions(attributions)

    score_vis = viz.VisualizationDataRecord(
                        word_attributions = attributions_sum,
                        pred_prob = torch.max(model(input_ids)[0]),
                        pred_class = torch.argmax(model(input_ids)[0]).cpu().numpy(),
                        true_class = true_class,
                        attr_class = text,
                        attr_score = attributions_sum.sum(),       
                        raw_input_ids = all_tokens,
                        convergence_score = delta)

    viz.visualize_text([score_vis])

<br>
<hr>
<br>
<br>

### Testes

In [27]:
text = "As pessoas não estão andando na estrada ao lado de uma bela cachoeira. Uma cachoeira está fluindo em uma piscina rasa"
true_class = 0
interpret_text(text, true_class)

In [28]:
# test id=8 
text = "Uma mulher está andando a cavalo. Não tem nenhuma mulher andando a cavalo"
true_class = 0
interpret_text(text, true_class)

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,0 (3.03),Uma mulher está andando a cavalo. Não tem nenhuma mulher andando a cavalo,2.55,[CLS] Uma mulher está anda ##ndo a cavalo . Não tem nenhuma mulher anda ##ndo a cavalo [SEP]


In [29]:
text = "Quatro pessoas estão andando através da neve espessa e o sol está se pondo. Ninguém está atravessando a neve da linda paisagem nevada"
true_class = 0
interpret_text(text, true_class)

In [30]:
text = "O cara está cuidadosamente alimentando a cobra com um rato. Um cara está alimentando um rato para a cobra"
true_class = 1
interpret_text(text, true_class)

In [31]:
text = "Estamos indo escalar uma montanha na europa. Vamos escalar uma montanha na america do sul"
true_class = 0
interpret_text(text, true_class)

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,1 (1.32),Estamos indo escalar uma montanha na europa. Vamos escalar uma montanha na america do sul,0.92,[CLS] Esta ##mos indo escala ##r uma montanha na europ ##a . Va ##mos escala ##r uma montanha na americ ##a do sul [SEP]


In [32]:
text = "O dia esta ensolarado e fresco aqui em Sao Carlos. A noite esta nublada e fria aqui em Santa Catarina"
true_class = 0
interpret_text(text, true_class)

In [33]:
text = "Hoje eu vou ir ao supermercado fazer algumas compras. Semana passada eu fui ao shopping fazer algumas compras"
true_class = 0
interpret_text(text, true_class)

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,0 (1.89),Hoje eu vou ir ao supermercado fazer algumas compras. Semana passada eu fui ao shopping fazer algumas compras,3.01,[CLS] Hoje eu vou ir ao supermer ##cado fazer algumas compras . Semana passada eu fu ##i ao sho ##pping fazer algumas compras [SEP]


In [34]:
# dataset de test 
text = "Um cachorro está correndo por um campo e está perseguindo uma bola. Um animal está indo em direção a um brinquedo"
true_class = 1 
interpret_text(text, true_class)

In [35]:
# dataset de test 
text = "Um lêmure está lambendo o dedo de uma pessoa. Um lêmure está mordendo o dedo de uma pessoa"
true_class = 0
interpret_text(text, true_class)

In [36]:
# dataset de test id="91"
text = "Uma vaqueira está montando um cavalo marrom e está dando a volta em um barril. Uma vaqueira está montando a cavalo"
true_class = 1
interpret_text(text, true_class)

In [37]:
# dataset de test id=105
text = "Um macaco está chutando uma pessoa. Um macaco está praticando artes marciais"
true_class = 0
interpret_text(text, true_class)

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,0 (2.88),Um macaco está chutando uma pessoa. Um macaco está praticando artes marciais,1.59,[CLS] Um maca ##co está chu ##tando uma pessoa . Um maca ##co está pratica ##ndo artes marciais [SEP]


In [38]:
# test proprio
text = "Eu tinha o habito de correr muito no passado. Nos dias atuais eu nao pratico nenhum esporte."
true_class = 0
interpret_text(text, true_class)

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,0 (2.64),Eu tinha o habito de correr muito no passado. Nos dias atuais eu nao pratico nenhum esporte.,2.37,[CLS] Eu tinha o habit ##o de correr muito no passado . Nos dias atuais eu na ##o pratic ##o nenhum esporte . [SEP]


In [39]:
# id=82 
text = "Vegetais verdes estão sendo misturados em uma panela por um homem. Um cara está misturando vegetais em uma panela"
true_class = 1 
interpret_text(text, true_class)

In [40]:
# id=86
text = "Uma mulher está cortando um limão. Um homem está cortando um limão"
true_class = 0
interpret_text(text, true_class)

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,0 (2.75),Uma mulher está cortando um limão. Um homem está cortando um limão,2.20,[CLS] Uma mulher está corta ##ndo um lim ##ão . Um homem está corta ##ndo um lim ##ão [SEP]


In [41]:
# id=94 
text = "Um cara está acariciando dois cachorros. Dois cachorros estão sendo acariciados por um cara"
true_class = 1 
interpret_text(text, true_class)

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,1 (2.43),Um cara está acariciando dois cachorros. Dois cachorros estão sendo acariciados por um cara,0.26,[CLS] Um cara está aca ##ric ##iando dois cachorro ##s . Dois cachorro ##s estão sendo aca ##ric ##iados por um cara [SEP]


In [42]:
# id=101 
text = "Dois homens estão lutando. Alguns homens estão batalhando"
true_class = 1 
interpret_text(text, true_class)

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,1 (2.45),Dois homens estão lutando. Alguns homens estão batalhando,1.41,[CLS] Dois homens estão lutando . Alguns homens estão batalha ##ndo [SEP]


In [43]:
# id=125
text = "O garoto está sendo treinado em artes marciais. O menino está praticando artes marciais"
true_class = 1 
interpret_text(text, true_class)

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,1 (2.45),O garoto está sendo treinado em artes marciais. O menino está praticando artes marciais,3.01,[CLS] O garoto está sendo treinado em artes marciais . O menino está pratica ##ndo artes marciais [SEP]


<br>
<br>
<hr>